In [16]:
import pandas as pd
import pickle

In [17]:
single_measures_df = pd.read_csv("predict_stock_prices.csv")

In [18]:
#prepare the dataset for prediction
#Drop Na valuesa
single_measures_df = single_measures_df.dropna()
#reset index
single_measures_df.reset_index(inplace=True)


In [19]:
weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday']
for weekday in weekdays:
    single_measures_df[weekday] = False
for weekday in weekdays:
    single_measures_df[weekday] = single_measures_df['weekday'] == weekday

In [20]:
#Getting weekday dummies
#weekday_dummies = pd.get_dummies(data = single_measures_df['weekday'])
#single_measures_df = pd.concat([single_measures_df, weekday_dummies], axis = 1)

In [21]:
single_measures_df

,index,Unnamed: 0,date,company,slope,r_squared,std_error,stdev,up_percentage,down_percentage,ticker,weekday,Monday,Tuesday,Wednesday,Thursday,Friday
0,0,0,2024-11-21,Afarak Group SE,-8.196616e-07,0.049180,4.732318e-07,0.000065,0.000000,1.000000,AFAGR.HE,Thursday,False,False,False,True,False
1,1,1,2024-11-21,Aktia Pankki Oyj,-5.954407e-04,0.412020,9.339987e-05,0.016201,0.050000,0.950000,AKTIA.HE,Thursday,False,False,False,True,False
2,2,2,2024-11-21,Alma Media Oyj,3.713522e-03,0.742706,2.869974e-04,0.075254,0.016667,0.983333,ALMA.HE,Thursday,False,False,False,True,False
3,3,3,2024-11-21,Anora Group Oyj,-2.629888e-04,0.425565,4.011995e-05,0.007041,0.066667,0.933333,ANORA.HE,Thursday,False,False,False,True,False
4,4,4,2024-11-21,Apetit Oyj,9.502631e-04,0.142540,3.060336e-04,0.043957,0.016667,0.983333,APETIT.HE,Thursday,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,111,111,2024-11-21,Wetteri Oyj,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.000000,WETTERI.HE,Thursday,False,False,False,True,False
94,112,112,2024-11-21,WithSecure Oyj,2.031094e-05,0.053466,1.122136e-05,0.001534,0.050000,0.950000,WITH.HE,Thursday,False,False,False,True,False
95,114,114,2024-11-21,Wärtsilä Oyj Abp,-3.446659e-03,0.896043,1.541512e-04,0.063589,0.150000,0.850000,WRT1V.HE,Thursday,False,False,False,True,False
96,115,115,2024-11-21,YIT Oyj,-1.542092e-04,0.211352,3.911423e-05,0.005858,0.050000,0.950000,YIT.HE,Thursday,False,False,False,True,False


In [22]:
#Drop unnecessary columns
company = single_measures_df['company'] #Saving company to be added later
single_measures_df = single_measures_df.drop(columns=['index','Unnamed: 0','date','company','ticker','weekday'])
single_measures_df

,slope,r_squared,std_error,stdev,up_percentage,down_percentage,Monday,Tuesday,Wednesday,Thursday,Friday
0,-8.196616e-07,0.049180,4.732318e-07,0.000065,0.000000,1.000000,False,False,False,True,False
1,-5.954407e-04,0.412020,9.339987e-05,0.016201,0.050000,0.950000,False,False,False,True,False
2,3.713522e-03,0.742706,2.869974e-04,0.075254,0.016667,0.983333,False,False,False,True,False
3,-2.629888e-04,0.425565,4.011995e-05,0.007041,0.066667,0.933333,False,False,False,True,False
4,9.502631e-04,0.142540,3.060336e-04,0.043957,0.016667,0.983333,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
93,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.000000,False,False,False,True,False
94,2.031094e-05,0.053466,1.122136e-05,0.001534,0.050000,0.950000,False,False,False,True,False
95,-3.446659e-03,0.896043,1.541512e-04,0.063589,0.150000,0.850000,False,False,False,True,False
96,-1.542092e-04,0.211352,3.911423e-05,0.005858,0.050000,0.950000,False,False,False,True,False


In [23]:
#rearrange columnt to match model
single_measures_df = single_measures_df[['slope','r_squared','std_error','stdev','up_percentage','down_percentage','Friday','Monday','Thursday','Tuesday','Wednesday']]
single_measures_df

,slope,r_squared,std_error,stdev,up_percentage,down_percentage,Friday,Monday,Thursday,Tuesday,Wednesday
0,-8.196616e-07,0.049180,4.732318e-07,0.000065,0.000000,1.000000,False,False,True,False,False
1,-5.954407e-04,0.412020,9.339987e-05,0.016201,0.050000,0.950000,False,False,True,False,False
2,3.713522e-03,0.742706,2.869974e-04,0.075254,0.016667,0.983333,False,False,True,False,False
3,-2.629888e-04,0.425565,4.011995e-05,0.007041,0.066667,0.933333,False,False,True,False,False
4,9.502631e-04,0.142540,3.060336e-04,0.043957,0.016667,0.983333,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
93,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.000000,False,False,True,False,False
94,2.031094e-05,0.053466,1.122136e-05,0.001534,0.050000,0.950000,False,False,True,False,False
95,-3.446659e-03,0.896043,1.541512e-04,0.063589,0.150000,0.850000,False,False,True,False,False
96,-1.542092e-04,0.211352,3.911423e-05,0.005858,0.050000,0.950000,False,False,True,False,False


In [24]:
#Loading models from pickle
with open('regression_model.pkl', 'rb') as f:
    reg_model = pickle.load(f)

with open('random_forest_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

with open('xgboost_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)   

In [25]:
#Making prediction
reg_predictions = reg_model.predict(single_measures_df)
rf_predictions = rf_model.predict(single_measures_df)
xgboost_predictions = xgb_model.predict(single_measures_df)

In [26]:
single_measures_df = single_measures_df.copy()
single_measures_df.loc[:, 'reg_predictions'] = reg_predictions
single_measures_df.loc[:, 'rf_predictions'] = rf_predictions
single_measures_df.loc[:, 'xgboost_predictions'] = xgboost_predictions
single_measures_df.loc[:, 'company'] = company


In [27]:
single_measures_df.sort_values(by=['reg_predictions'], ascending = False)

,slope,r_squared,std_error,stdev,up_percentage,down_percentage,Friday,Monday,Thursday,Tuesday,Wednesday,reg_predictions,rf_predictions,xgboost_predictions,company
28,0.009365,0.242096,0.002176,0.332406,0.033333,0.966667,False,False,True,False,False,0.020020,0.007042,0.005176,Gofore Oyj
29,0.006807,0.499050,0.000896,0.168291,0.166667,0.833333,False,False,True,False,False,0.013428,0.009059,0.041234,Harvia Oyj
67,0.004646,0.565470,0.000535,0.107895,0.033333,0.966667,False,False,True,False,False,0.008789,0.009916,0.017393,Raute Oyj
38,0.003176,0.095027,0.001287,0.179925,0.150000,0.850000,False,False,True,False,False,0.008301,0.007015,0.058313,Konecranes Plc
71,0.004141,0.687198,0.000367,0.087248,0.283333,0.716667,False,False,True,False,False,0.008301,0.009916,0.014307,Sampo Oyj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,-0.002770,0.909111,0.000115,0.050730,0.016667,0.983333,False,False,True,False,False,-0.003174,-0.012348,0.004024,Lassila & Tikanoja Oyj
95,-0.003447,0.896043,0.000154,0.063589,0.150000,0.850000,False,False,True,False,False,-0.003662,-0.012348,-0.016026,Wärtsilä Oyj Abp
31,-0.003509,0.648613,0.000339,0.076099,0.166667,0.833333,False,False,True,False,False,-0.003662,-0.009536,-0.009040,Huhtamäki Oyj
13,-0.006763,0.490007,0.000906,0.168735,0.266667,0.733333,False,False,True,False,False,-0.006836,-0.009210,-0.000732,Cargotec Corporation


In [28]:
single_measures_df.sort_values(by=['reg_predictions'], ascending = False).head(10)

,slope,r_squared,std_error,stdev,up_percentage,down_percentage,Friday,Monday,Thursday,Tuesday,Wednesday,reg_predictions,rf_predictions,xgboost_predictions,company
28,0.009365,0.242096,0.002176,0.332406,0.033333,0.966667,False,False,True,False,False,0.020020,0.007042,0.005176,Gofore Oyj
29,0.006807,0.499050,0.000896,0.168291,0.166667,0.833333,False,False,True,False,False,0.013428,0.009059,0.041234,Harvia Oyj
67,0.004646,0.565470,0.000535,0.107895,0.033333,0.966667,False,False,True,False,False,0.008789,0.009916,0.017393,Raute Oyj
38,0.003176,0.095027,0.001287,0.179925,0.150000,0.850000,False,False,True,False,False,0.008301,0.007015,0.058313,Konecranes Plc
71,0.004141,0.687198,0.000367,0.087248,0.283333,0.716667,False,False,True,False,False,0.008301,0.009916,0.014307,Sampo Oyj
2,0.003714,0.742706,0.000287,0.075254,0.016667,0.983333,False,False,True,False,False,0.007080,0.011694,0.019321,Alma Media Oyj
70,0.003220,0.422834,0.000494,0.086475,0.100000,0.900000,False,False,True,False,False,0.006592,0.008068,0.049564,Revenio Group Oyj
26,0.002657,0.810863,0.000169,0.051539,0.350000,0.650000,False,False,True,False,False,0.005615,0.010896,0.026511,Fortum Oyj
20,0.001548,0.271809,0.000333,0.051862,0.200000,0.800000,False,False,True,False,False,0.003418,0.006563,-0.000744,Elisa Oyj
15,0.001558,0.769591,0.000112,0.031020,0.033333,0.966667,False,False,True,False,False,0.003174,0.010807,0.027498,Componenta Corporation


In [29]:
single_measures_df.sort_values(by=['reg_predictions'], ascending = False).tail(10)

,slope,r_squared,std_error,stdev,up_percentage,down_percentage,Friday,Monday,Thursday,Tuesday,Wednesday,reg_predictions,rf_predictions,xgboost_predictions,company
62,-0.001200,0.720200,0.000098,0.024702,0.000000,1.000000,False,False,True,False,False,-0.001221,-0.011668,-0.014934,Purmo Group Oyj
92,-0.002701,0.270075,0.000583,0.090759,0.000000,1.000000,False,False,True,False,False,-0.002441,-0.008234,-0.017405,Viking Line Abp
55,-0.002614,0.484741,0.000354,0.065564,0.133333,0.866667,False,False,True,False,False,-0.002441,-0.009067,-0.028749,Orion Oyj
86,-0.002754,0.832624,0.000162,0.052706,0.166667,0.833333,False,False,True,False,False,-0.002686,-0.012348,0.001584,Tokmanni Group Oyj
41,-0.002797,0.563532,0.000323,0.065081,0.033333,0.966667,False,False,True,False,False,-0.002930,-0.009067,-0.003168,Marimekko Oyj
40,-0.002770,0.909111,0.000115,0.050730,0.016667,0.983333,False,False,True,False,False,-0.003174,-0.012348,0.004024,Lassila & Tikanoja Oyj
95,-0.003447,0.896043,0.000154,0.063589,0.150000,0.850000,False,False,True,False,False,-0.003662,-0.012348,-0.016026,Wärtsilä Oyj Abp
31,-0.003509,0.648613,0.000339,0.076099,0.166667,0.833333,False,False,True,False,False,-0.003662,-0.009536,-0.009040,Huhtamäki Oyj
13,-0.006763,0.490007,0.000906,0.168735,0.266667,0.733333,False,False,True,False,False,-0.006836,-0.009210,-0.000732,Cargotec Corporation
64,-0.009514,0.685839,0.000845,0.200628,0.116667,0.883333,False,False,True,False,False,-0.010986,-0.009842,-0.008966,Qt Group Oyj


In [ ]:
top10_reg = single_measures_df[['company','reg_predictions','rf_predictions','xgboost_predictions']].sort_values(by=['reg_predictions'], ascending=False).head(10)
top10_rf = single_measures_df[['company','reg_predictions','rf_predictions','xgboost_predictions']].sort_values(by=['rf_predictions'], ascending=False).head(10)
top10_xgb = single_measures_df[['company','reg_predictions','rf_predictions','xgboost_predictions']].sort_values(by=['xgboost_predictions'], ascending=False).head(10)

In [59]:
bottom10_reg = single_measures_df[['company','reg_predictions','rf_predictions','xgboost_predictions']].sort_values(by=['reg_predictions'], ascending=False).tail(10)
bottom10_rf = single_measures_df[['company','reg_predictions','rf_predictions','xgboost_predictions']].sort_values(by=['rf_predictions'], ascending=False).tail(10)
bottom10_xgb = single_measures_df[['company','reg_predictions','rf_predictions','xgboost_predictions']].sort_values(by=['xgboost_predictions'], ascending=False).tail(10)

In [ ]:
tops = pd.DataFrame([top10_reg['company'], top10_rf['company'], top10_xgb['company']]).transpose()
tops.columns = ['regression','random_forest','xbgoost']
bottoms = pd.DataFrame([bottom10_reg['company'], bottom10_rf['company'], bottom10_xgb['company']]).transpose()
bottoms.columns = ['regression','random_forest','xbgoost']

In [57]:
tops.value_counts()

regression              random_forest           xbgoost               
Alma Media Oyj          Alma Media Oyj          Alma Media Oyj            1
Componenta Corporation  Componenta Corporation  Componenta Corporation    1
Fortum Oyj              Fortum Oyj              Fortum Oyj                1
Harvia Oyj              Harvia Oyj              Harvia Oyj                1
Name: count, dtype: int64

In [58]:
tops

,regression,random_forest,xbgoost
28,Gofore Oyj,NaN,NaN
29,Harvia Oyj,Harvia Oyj,Harvia Oyj
67,Raute Oyj,Raute Oyj,NaN
38,Konecranes Plc,NaN,Konecranes Plc
71,Sampo Oyj,Sampo Oyj,NaN
2,Alma Media Oyj,Alma Media Oyj,Alma Media Oyj
70,Revenio Group Oyj,NaN,Revenio Group Oyj
26,Fortum Oyj,Fortum Oyj,Fortum Oyj
20,Elisa Oyj,NaN,NaN
15,Componenta Corporation,Componenta Corporation,Componenta Corporation
